# Compiling Circuits for EeroQ via Cirq

## Import Requirements

This tutorial will showcase how to compile a circuit for EeroQ hardware using the ```cirq-superstaq``` client. 

In [1]:
# Required imports
try:
    import cirq
    import cirq_superstaq as css
except ImportError:
    print("Installing cirq-superstaq...")
    %pip install --quiet 'cirq-superstaq[examples]'
    print("Installed cirq-superstaq.")
    print("You may need to restart the kernel to import newly installed packages.")
    import cirq
    import cirq_superstaq as css

try:
    import qiskit
    import qiskit_superstaq as qss
except ImportError:
    print("Installing qiskit-superstaq...")
    %pip install --quiet 'qiskit-superstaq[examples]'
    print("Installed qiskit-superstaq.")
    print("You may need to restart the kernel to import newly installed packages.")
    import qiskit
    import qiskit_superstaq as qss

# Optional imports
import numpy as np
import os  # Used if setting a token as an environment variable

%load_ext autoreload
%autoreload 2

To interface Superstaq via Cirq, we must first instantiate a service provider in ```cirq-superstaq``` with ```Service()```. We then supply a Superstaq API key (which you can get from https://superstaq.infleqtion.com) by either providing the API key as an argument of Service, i.e., ```css.Service(api_key="token")```, or by setting it as an environment variable. (see more details [here](https://superstaq.readthedocs.io/en/latest/get_started/basics/basics_css.html#Set-up-access-to-Superstaq%E2%80%99s-API)).

In [88]:
service = css.Service()

# EeroQ Gates

In [3]:
dd_gate = css.DDPowGate(exponent=1)
cirq.Circuit(dd_gate.on(cirq.q(0), cirq.q(1)))

0: ───DD───
      │
1: ───DD───

In [4]:
cirq.unitary(dd_gate)

array([[ 0.-1.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.-1.j]])

Below is the EeroQ protocol for a CZ Gate.

![title](images/DD_gate.png)

In [5]:
circuit = cirq.Circuit(cirq.CZ(cirq.q(0), cirq.q(1)))
compiled_circuit = service.compile(circuit, "eeroq_wonderlake_qpu").circuit
compiled_circuit

0+: ───────│──────────────────────────────
           │
0-: ───X───│───DD────────S──────DD────────
           │   │                │
1+: ───────│───┼─────────Z──────┼─────────
           │   │                │
1-: ───X───│───DD^0.25───S^-1───DD^0.25───

In [6]:
def compute_unitary(circuit: cirq.Circuit):
    """Computes the n*n unitary of a 2n electron EeroQ circuit"""
    unitary = cirq.unitary(circuit[1:]).reshape((4,) * cirq.num_qubits(circuit))
    mat = unitary[tuple(slice(1, 3) for _ in range(cirq.num_qubits(circuit)))]
    dim = round(np.sqrt(mat.size))
    mat = mat.reshape(dim, dim)
    return mat

In [7]:
mat = compute_unitary(compiled_circuit)
mat/mat[0][0]

array([[ 1.-0.00000000e+00j, -0.+0.00000000e+00j, -0.+0.00000000e+00j,
        -0.+0.00000000e+00j],
       [-0.+0.00000000e+00j,  1.+7.85046229e-17j, -0.+0.00000000e+00j,
        -0.+0.00000000e+00j],
       [-0.+0.00000000e+00j, -0.+0.00000000e+00j,  1.+7.85046229e-17j,
        -0.+0.00000000e+00j],
       [-0.+0.00000000e+00j, -0.+0.00000000e+00j, -0.+0.00000000e+00j,
        -1.-0.00000000e+00j]])

In [8]:
cirq.allclose_up_to_global_phase(cirq.unitary(circuit), mat)

True

In [9]:
circuit += cirq.measure(cirq.q(0), cirq.q(1))
job = service.create_job(circuit, target= "eeroq_wonderlake_qpu", repetitions = 10, method="dry-run")

In [10]:
job.counts()

[{'00': 10}]

# Circuit Compilation

In [11]:
qubits = cirq.LineQubit.range(2)
circuit = cirq.Circuit(cirq.H(qubits[0]), cirq.CNOT(qubits[0], qubits[1]), cirq.measure(*qubits))
circuit

0: ───H───@───M───
          │   │
1: ───────X───M───

In [12]:
compiled_circuit = service.compile(circuit, "eeroq_wonderlake_qpu").circuit
compiled_circuit

0+: ───────│───S^-1───│───DD────────S──────│─────────────Z────────────────│───DD────────Z──────│───M('q(0),q(1)')───
           │          │   │                │                              │   │                │   │
0-: ───X───│──────────│───DD^-0.5──────────│───DD────────S──────DD────────│───DD───────────────│───┼────────────────
           │          │                    │   │                │         │                    │   │
1+: ───────│───S──────│───DD────────S^-1───│───┼─────────S^-1───┼─────────│───DD────────S^-1───│───M────────────────
           │          │   │                │   │                │         │   │                │
1-: ───X───│──────────│───DD^-0.5──────────│───DD^0.25───S^-1───DD^0.25───│───DD^-0.5──────────│────────────────────

In [6]:
gate_domain = {
    cirq.X: 1,
    cirq.Y: 1,
    cirq.Z: 1,
    cirq.S: 1,
    cirq.T: 1,
    cirq.H: 1,
    cirq.rx(1.23): 1,
    cirq.ry(2.34): 1,
    cirq.CZ: 2,
    cirq.CX: 2,
    cirq.CX**0.5: 2,
    cirq.SWAP: 2,
    cirq.ISWAP: 2,
    css.ZZSwapGate(1.23): 2,
    css.Barrier(3): 3,
}

In [7]:
n, depth, op_density = (4, 8, 0.8)
qubits = cirq.LineQubit.range(n)
circuit = cirq.testing.random_circuit(qubits, depth, op_density, gate_domain=gate_domain)
circuit

0: ───────Rx(0.392π)───T────────────X────────────────Y───────────────
                                    │
1: ─────────────────────────────────@───S────────────iSwap───────────
                                                     │
2: ───×────────────────Rx(0.392π)───────Ry(0.745π)───iSwap───────Z───
      │
3: ───×───Rx(0.392π)─────────────────────────────────────────────Z───

In [89]:
circuit.insert(depth // 2, css.barrier(*qubits))
compiled_circuit = service.compile(circuit, "eeroq_wonderlake_qpu").circuit
compiled_circuit

0+: ───────│───Z^-0.397───│───DD──────────Z^0.397─────│─────────────Z^0.251──────────────│───DD────────Z^-0.251───│─────────────Z^-0.249─────────────│────────────────────────│──────────────────────────────────│────────────────────────────│───│───S^-1──────│───DD─────────S──────────│─────────────────────────────────│───────────────────────│──────────────────────────────────│───────────────────────
           │              │   │                       │                                  │   │                    │                                  │                        │                                  │                            │   │             │   │                     │                                 │                       │                                  │
0-: ───X───│──────────────│───DD^-0.732───────────────│───DD────────S──────────DD────────│───DD^-0.5──────────────│──────────────────────────────────│────────────────────────│──────────────────────────────────│────────────────────────────│───│─────────────│───DD────────────────────│─────────────────────────────────│───────────────────────│──────────────────────────────────│───────────────────────
           │              │                           │   │                    │         │                        │                                  │                        │                                  │                            │   │             │                         │                                 │                       │                                  │
1+: ───────│──────────────│───────────────────────────│───┼────────────────────┼─────────│────────────────────────│──────────────────────────────────│────────────────────────│──────────────────────────────────│────────────────────────────│───│───T^-1──────│───DD─────────T──────────│─────────────Z^0.75──────────────│───DD────────Z^-0.75───│─────────────T^-1─────────────────│───DD────────T─────────
           │              │                           │   │                    │         │                        │                                  │                        │                                  │                            │   │             │   │                     │                                 │   │                   │                                  │   │
1-: ───X───│──────────────│───────────────────────────│───DD^0.25───S^-1───────DD^0.25───│────────────────────────│──────────────────────────────────│────────────────────────│──────────────────────────────────│────────────────────────────│───│─────────────│───DD^-0.5───────────────│───DD────────S─────────DD────────│───DD^-0.5─────────────│───DD────────S──────────DD────────│───DD^-0.5─────────────
           │              │                           │                                  │                        │                                  │                        │                                  │                            │   │             │                         │   │                   │         │                       │   │                    │         │
2+: ───────│───Z^0.589────│───DD──────────Z^-0.589────│─────────────Z^-0.768─────────────│───DD────────Z^0.768────│─────────────Z^0.232──────────────│───DD────────Z^-0.232───│─────────────Z^(-1/9)─────────────│───DD────────────Z^-0.976───│───│───Z^0.807───│───DD─────────Z^-0.807───│───┼─────────Z^0.007───┼─────────│───DD────────Z^0.993───│───┼─────────Z^-0.993───┼─────────│───DD────────Z^-0.75───
           │              │   │                       │                                  │   │                    │                                  │   │                    │                                  │   │                        │   │             │   │                     │   │                   │         │   │                   │   │                    │         │   │
2-: ───X───│──────────────│───DD^-0.491───────────────│───DD────────S──────────DD────────│───DD^-0.5───────

In [16]:
mat = compute_unitary(compiled_circuit)
cirq.testing.assert_allclose_up_to_global_phase(cirq.unitary(circuit), mat, atol=1e-8)

In [79]:
service = qss.superstaq_provider.SuperstaqProvider(remote_host="https://127.0.0.1:5000")


In [80]:
circuit1 = qiskit.QuantumCircuit(1, 1)

circuit1.h(0)
circuit1.measure(0, 0)
circuit1.draw(fold=-1)

┌───┐┌─┐
  q: ┤ H ├┤M├
     └───┘└╥┘
c: 1/══════╩═
           0

In [81]:
backend = service.get_backend("eeroq_wonderlake_qpu")

In [86]:
circuit1 = ss.converters.cirq_to_qiskit(circuit, circuit.all_qubits())
backend.compile(circuit1)

0: ───────Rx(0.392π)───T────────────X───│────────────────Y───────────────
                                    │   │
1: ─────────────────────────────────@───│───S────────────iSwap───────────
                                        │                │
2: ───×────────────────Rx(0.392π)───────│───Ry(0.745π)───iSwap───────Z───
      │                                 │
3: ───×───Rx(0.392π)────────────────────│────────────────────────────Z───
{cirq.LineQubit(0): 0, cirq.LineQubit(3): 1, cirq.LineQubit(1): 2, cirq.LineQubit(2): 3}


/home/palash/cq-env/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


CompilerOutput(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fa4d12d3010>, {0: 0, 1: 2, 2: 4, 3: 6}, {0: 0, 1: 2, 2: 4, 3: 6}, None, None, None)

In [87]:
backend.compile(circuit1).circuit.draw()

/home/palash/cq-env/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


░ ┌─────────────┐ ░ ┌──────────────┐ ┌────────────┐  ░ »
 q_0: ──────░─┤ Rz(-1.2482) ├─░─┤0             ├─┤ Rz(1.2482) ├──░─»
      ┌───┐ ░ └─────────────┘ ░ │  Dd(-2.3002) │ └────────────┘  ░ »
 q_1: ┤ X ├─░─────────────────░─┤1             ├─────────────────░─»
      └───┘ ░  ┌────────────┐ ░ ├──────────────┤┌─────────────┐  ░ »
 q_2: ──────░──┤ Rz(0.9135) ├─░─┤0             ├┤ Rz(-0.9135) ├──░─»
      ┌───┐ ░  └────────────┘ ░ │  Dd(-1.0612) │└─────────────┘  ░ »
 q_3: ┤ X ├─░─────────────────░─┤1             ├─────────────────░─»
      └───┘ ░                 ░ └──────────────┘                 ░ »
 q_4: ──────░─────────────────░──────────────────────────────────░─»
      ┌───┐ ░                 ░                                  ░ »
 q_5: ┤ X ├─░─────────────────░──────────────────────────────────░─»
      └───┘ ░ ┌─────────────┐ ░ ┌──────────────┐┌──────────────┐ ░ »
 q_6: ──────░─┤ Rz(0.79035) ├─░─┤0             ├┤ Rz(-0.79035) ├─░─»
      ┌───┐ ░ └─────────────┘ ░ │  Dd(-1.8594) │└──────────────┘ ░ »
 q_7: ┤ X ├─░─────────────────░─┤1             ├─────────────────░─»
      └───┘ ░                 ░ └──────────────┘                 ░ »
 q_8: ──────░──────────────────────────────────────────────────────»
      ┌───┐ ░                                                      »
 q_9: ┤ X ├─░──────────────────────────────────────────────────────»
      └───┘ ░                                                      »
q_10: ──────░──────────────────────────────────────────────────────»
      ┌───┐ ░                                                      »
q_11: ┤ X ├─░──────────────────────────────────────────────────────»
      └───┘ ░                                                      »
q_12: ──────░──────────────────────────────────────────────────────»
      ┌───┐ ░                                                      »
q_13: ┤ X ├─░──────────────────────────────────────────────────────»
      └───┘ ░                                                      »
q_14: ──────░──────────────────────────────────────────────────────»
      ┌───┐ ░                                                      »
q_15: ┤ X ├─░──────────────────────────────────────────────────────»
      └───┘ ░                                                      »
q_16: ──────░──────────────────────────────────────────────────────»
      ┌───┐ ░                                                      »
q_17: ┤ X ├─░──────────────────────────────────────────────────────»
      └───┘ ░                                                      »
q_18: ──────░──────────────────────────────────────────────────────»
      ┌───┐ ░                                                      »
q_19: ┤ X ├─░──────────────────────────────────────────────────────»
      └───┘ ░                                                      »
«      ┌─────────────┐                                              ░ »
« q_0: ┤ Rz(0.78743) ├──────────────────────────────────────────────░─»
«      └─────────────┘ ┌──────────┐  ┌───┐             ┌──────────┐ ░ »
« q_1: ────────────────┤0         ├──┤ S ├─────────────┤0         ├─░─»
«      ┌─────────────┐ │          │  └───┘             │          │ ░ »
« q_2: ┤ Rz(-2.1513) ├─┤          ├────────────────────┤          ├─░─»
«      └─┬──────────┬┘ │          │  ┌───┐ ┌──────────┐│          │ ░ »
« q_3: ──┤0         ├──┤  Dd(π/4) ├──┤ S ├─┤0         ├┤  Dd(π/4) ├─░─»
«        │          │  │          │  └───┘ │          ││          │ ░ »
« q_4: ──┤          ├──┤          ├────────┤          ├┤          ├─░─»
«        │          │  │          │ ┌─────┐│          ││          │ ░ »
« q_5: ──┤  Dd(π/4) ├──┤1         ├─┤ Sdg ├┤  Dd(π/4) ├┤1         ├─░─»
«        │          │ ┌┴──────────┴┐└─────┘│          │└──────────┘ ░ »
« q_6: ──┤          ├─┤ Rz(2.9673) ├───────┤          ├─────────────░─»
«        │          │ └──┬─────┬───┘       │          │             ░ »
« q_7: ──┤1         ├────┤ Sdg ├───────────┤1         ├─────────────░─»
«        └──────────┘    └─────

In [26]:
compiled_circuit

0+: ───────│───Z^-0.737───│───DD──────────Z^0.737───│────────────────────────────────│───────────────────────│────────────────────────────────│─────────────────────────│─────────────────────────────────│───────────────────────────────│──────────────────────────│─────────────Z^-0.955─────────────│───DD────────Z^0.955───│─────────────Z^0.045──────────────│───DD────────Z^-0.045───│─────────────Z^0.14───────────────│───DD──────────Z^0.029─────│───│────────────│───────────────────────│──────────────────────────────│───────────────────────│────────────────────────────────│───────────────────────│──────────────────────────────│───────────────────
           │              │   │                     │                                │                       │                                │                         │                                 │                               │                          │                                  │   │                   │                                  │   │                    │                                  │   │                       │   │            │                       │                              │                       │                                │                       │                              │
0-: ───X───│──────────────│───DD^-0.144─────────────│────────────────────────────────│───────────────────────│────────────────────────────────│─────────────────────────│─────────────────────────────────│───────────────────────────────│──────────────────────────│───DD────────S──────────DD────────│───DD^-0.5─────────────│───DD────────S──────────DD────────│───DD^-0.5──────────────│───DD────────S──────────DD────────│───DD^-0.39────────────────│───│────────────│───────────────────────│──────────────────────────────│───────────────────────│────────────────────────────────│───────────────────────│──────────────────────────────│───────────────────
           │              │                         │                                │                       │                                │                         │                                 │                               │                          │   │                    │         │                       │   │                    │         │                        │   │                    │         │                           │   │            │                       │                              │                       │                                │                       │                              │
1+: ───────│───Z^-0.75────│───DD──────────Z^0.75────│─────────────Z^0.75─────────────│───DD────────Z^-0.75───│─────────────Z^0.75─────────────│───DD──────────T─────────│─────────────T^-1────────────────│───DD────────T───│───Z^0.737───│───DD──────────Z^-0.737───│───┼─────────Z^0.519────┼─────────│───DD────────Z^0.481───│───┼─────────Z^-0.481───┼─────────│───DD────────Z^-0.519───│───┼─────────Z^-0.116───┼─────────│───DD──────────Z^(-1/16)───┼───│───Z^0.75───│───DD────────Z^-0.75───│─────────────T^-1─────────────│───DD────────T─────────│─────────────Z^0.75─────────────│───DD────────Z^-0.75───│──────────────────────────────│───────────────S───
           │              │   │                     │                                │   │                   │                                │   │                     │                                 │   │             │             │   │                      │   │                    │         │   │                   │   │                    │         │   │                    │   │                    │         │   │                       │   │            │   │                   │                              │   │                   │                                │   │                   │                              │
1-: ───X───│──────────────│───DD^-0.5───────────────│───DD────────S────────DD────────│───DD^-0.5─────────────│───DD────────S^-1─────DD────────│───DD^-0.5───────────────│

In [2]:
import superstaq as ss

In [29]:
ss.converters.cirq_to_qiskit(compiled_circuit, compiled_circuit.all_qubits())

{cirq.NamedQubit('1-'): 0, cirq.NamedQubit('3-'): 1, cirq.NamedQubit('2-'): 2, cirq.NamedQubit('0-'): 3, cirq.NamedQubit('0+'): 4, cirq.NamedQubit('3+'): 5, cirq.NamedQubit('1+'): 6, cirq.NamedQubit('2+'): 7} !!!!!!!!!!!!!!!!!!!!!!!!


In [64]:
device = ss.devices.eeroq.EeroQDevice(10, "test")

In [71]:
lq = cirq.LineQubit.range(10)
qm = dict(zip(lq, device._qubits[::2]))
for l, p in qm.items():
    print(p)

00+
01+
02+
03+
04+
05+
06+
07+
08+
09+


In [74]:
device.compile(circuit).circuit

00+: ───────│───Z^-0.397───│───DD──────────Z^0.397────│─────────────Z^0.251──────────────│───DD────────Z^-0.251───│─────────────Z^-0.249─────────────│────────────────────────│──────────────────────────────────│───────────────────────────│───S^-1──────│───DD─────────S──────────│─────────────────────────────────│───────────────────────│──────────────────────────────────│───────────────────────
            │              │   │                      │                                  │   │                    │                                  │                        │                                  │                           │             │   │                     │                                 │                       │                                  │
00-: ───X───│──────────────│───DD^-0.732──────────────│───DD────────S──────────DD────────│───DD^-0.5──────────────│──────────────────────────────────│────────────────────────│──────────────────────────────────│───────────────────────────│─────────────│───DD────────────────────│─────────────────────────────────│───────────────────────│──────────────────────────────────│───────────────────────
            │              │                          │   │                    │         │                        │                                  │                        │                                  │                           │             │                         │                                 │                       │                                  │
01+: ───────│──────────────│──────────────────────────│───┼────────────────────┼─────────│────────────────────────│──────────────────────────────────│────────────────────────│──────────────────────────────────│───────────────────────────│───T^-1──────│───DD─────────T──────────│─────────────Z^0.75──────────────│───DD────────Z^-0.75───│─────────────T^-1─────────────────│───DD────────T─────────
            │              │                          │   │                    │         │                        │                                  │                        │                                  │                           │             │   │                     │                                 │   │                   │                                  │   │
01-: ───X───│──────────────│──────────────────────────│───DD^0.25───S^-1───────DD^0.25───│────────────────────────│──────────────────────────────────│────────────────────────│──────────────────────────────────│───────────────────────────│─────────────│───DD^-0.5───────────────│───DD────────S─────────DD────────│───DD^-0.5─────────────│───DD────────S──────────DD────────│───DD^-0.5─────────────
            │              │                          │                                  │                        │                                  │                        │                                  │                           │             │                         │   │                   │         │                       │   │                    │         │
02+: ───────│───Z^0.291────│───DD──────────Z^-0.291───│─────────────Z^-0.685─────────────│───DD────────Z^0.685────│─────────────Z^0.315──────────────│───DD────────Z^-0.315───│─────────────Z^-0.374─────────────│───DD───────────Z^-0.745───│───Z^0.807───│───DD─────────Z^-0.807───│───┼─────────Z^0.007───┼─────────│───DD────────Z^0.993───│───┼─────────Z^-0.993───┼─────────│───DD────────Z^-0.75───
            │              │   │                      │                                  │   │                    │                                  │   │                    │                                  │   │                       │             │   │                     │   │                   │         │   │                   │   │                    │         │   │
02-: ───X───│──────────────│───DD^-0.338──────────────│───DD────────S──────────DD────────│───DD^-0.5──────────────│───DD────────S──────────DD────────│───DD^-

In [21]:
device._qubits

[cirq.NamedQubit('00+'),
 cirq.NamedQubit('00-'),
 cirq.NamedQubit('01+'),
 cirq.NamedQubit('01-'),
 cirq.NamedQubit('02+'),
 cirq.NamedQubit('02-'),
 cirq.NamedQubit('03+'),
 cirq.NamedQubit('03-'),
 cirq.NamedQubit('04+'),
 cirq.NamedQubit('04-'),
 cirq.NamedQubit('05+'),
 cirq.NamedQubit('05-'),
 cirq.NamedQubit('06+'),
 cirq.NamedQubit('06-'),
 cirq.NamedQubit('07+'),
 cirq.NamedQubit('07-'),
 cirq.NamedQubit('08+'),
 cirq.NamedQubit('08-'),
 cirq.NamedQubit('09+'),
 cirq.NamedQubit('09-')]